In [1]:
import pandas as pd
import numpy as np 
from copy import deepcopy
from util.dataloader import cached_bison_data, load_pump_curves, cached_site_info, fetch_bison_data
from util.preprocessing import *
from util.calculations import *
from util.format_calculations import *
import plotly.express as px

# fetch data
# filepath = "/Users/audreyder/Neuralix/syncdatabase_2025-02-20.csv"
# data = fetch_bison_data(filepath)

# load data
filepath = "/Users/audreyder/Neuralix/syncdatabase_2025-02-20.csv"
data = cached_bison_data(filepath) 

# filter data / preprocess
data = threshold_filtering(data) 

site_ids = [33404, 33467, 57740, 33614]
site_pump_curves = load_pump_curves(site_ids) # load pump curve data
sites_info = cached_site_info(dict=True)
# FUTURE PROOF NOTE: eventually, there will be many sites with many pumps, not just four sites with one pump each

Adding Datetime Timestamp and Delta T columns...
Data columns: Index(['site_id', 'pump_id', 'timestamp', 'facility_name', 'vibration',
       'thrust temperature', 'suction pressure', 'discharge pressure',
       'flow rate', 'frequency', 'amps', 'ampsa', 'ampsb', 'ampsc', 'volts',
       'voltsa', 'voltsb', 'voltsc', 'meter total', 'timestamp_datetime',
       'delta_t'],
      dtype='object')
Number of rows: 261571
Earliest timestamp: 2024-11-22 12:55:10
Latest timestamp: 2025-02-20 12:53:04


/Users/audreyder/Neuralix/audrey-workspace/Bison_Water/util/preprocessing.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[frequency_int_column] = df[frequency_float_column].round().astype('Int64')


In [2]:
kpis_all_sites = {site_id:None for site_id in site_ids}
site_pump_curves = load_pump_curves(site_ids)
for site_id in site_ids: # for each site...
    site_data = deepcopy(data[data['site_id']==site_id]) # Looks at individual site data
    pump_curve_data = site_pump_curves[site_id]
    site_data, site_pump_kpis, sampling_interval = compute_kpis_for_sites(site_data, site_id,kpis=['kWh/BBL','Flow Rate','perc_from_BEP','norm_perc_from_BEP'],
                                                                          site_pump_curves=pump_curve_data,sites_info=sites_info)
    site_name = sites_info[site_id]['site_name']
    kpis_all_sites[site_id] = site_pump_kpis

Recorded frequencies outside calibration range (KPI is not normalized): set()
Interpolated frequencies within calibration range (KPI normalization is approximated): {47, 49, 51, 53, 55}
Normalized frequencies: {46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56}
Recorded frequencies outside calibration range (KPI is not normalized): set()
Interpolated frequencies within calibration range (KPI normalization is approximated): {48, 50, 52, 54, 56}
Normalized frequencies: {47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57}
Recorded frequencies outside calibration range (KPI is not normalized): set()
Interpolated frequencies within calibration range (KPI normalization is approximated): {45, 46, 48, 50, 52, 54, 56, 58}
Normalized frequencies: {44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59}
No intersection found between speed line 65 and efficiency line Min
Missing intersection points for speed line 65, skipping
Recorded frequencies outside calibration range (KPI is not normalized): set()
I

In [3]:
df_all, game_kpi = kpi_charts(kpis_all_sites, site_ids, sites_info, kpis=['kWh/BBL','Flow Rate'],print_chart=True)

Union City 2 SWD


,Flow Rate,kWh/BBL,perc_from_BEP,abs_perc_from_BEP,norm_perc_from_BEP,abs_norm_perc_from_BEP,site_id
47.0,9274.486,0.792,-98.069,98.069,-1.344,1.344,Union City 2 SWD
48.0,9353.401,0.822,-99.475,99.475,-2.769,2.769,Union City 2 SWD
49.0,10624.673,0.787,-88.621,88.621,-0.926,0.926,Union City 2 SWD
50.0,11276.133,0.792,-81.144,81.144,-0.750,0.750,Union City 2 SWD
51.0,12228.688,0.783,-68.979,68.979,-0.196,0.196,Union City 2 SWD
52.0,12781.712,0.792,-63.507,63.507,-1.241,1.241,Union City 2 SWD
53.0,13651.880,0.787,-52.980,52.980,-0.550,0.550,Union City 2 SWD
54.0,14416.753,0.795,-44.305,44.305,-0.459,0.459,Union City 2 SWD
55.0,15141.788,0.798,-36.393,36.393,-0.964,0.964,Union City 2 SWD
57.0,16916.667,0.802,-16.246,16.246,NaN,NaN,Union City 2 SWD


Siegrist SWD


/Users/audreyder/Neuralix/audrey-workspace/Bison_Water/util/format_calculations.py:80: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat([df_all,df_site])


,Flow Rate,kWh/BBL,perc_from_BEP,abs_perc_from_BEP,norm_perc_from_BEP,abs_norm_perc_from_BEP,site_id
47.0,11968.783,0.683,-37.888,37.888,-3.614,3.614,Siegrist SWD
48.0,12149.354,0.685,-38.609,38.609,-5.250,5.250,Siegrist SWD
49.0,12724.716,0.697,-33.577,33.577,-7.492,7.492,Siegrist SWD
50.0,13328.427,0.714,-28.208,28.208,-4.553,4.553,Siegrist SWD
51.0,13549.448,0.734,-28.558,28.558,-4.530,4.530,Siegrist SWD
52.0,14692.218,0.752,-15.195,15.195,-1.707,1.707,Siegrist SWD
53.0,14791.354,0.757,-17.537,17.537,-6.651,6.651,Siegrist SWD
54.0,15628.387,0.770,-8.872,8.872,-1.321,1.321,Siegrist SWD
55.0,16196.782,0.764,-4.401,4.401,-0.156,0.156,Siegrist SWD
56.0,16361.398,0.786,-5.567,5.567,-1.820,1.820,Siegrist SWD


Canadian SWD


,Flow Rate,kWh/BBL,perc_from_BEP,abs_perc_from_BEP,norm_perc_from_BEP,abs_norm_perc_from_BEP,site_id
42.0,9076.182,0.606,-63.764,63.764,NaN,NaN,Canadian SWD
43.0,9393.030,0.624,-62.348,62.348,NaN,NaN,Canadian SWD
44.0,10429.541,0.617,-50.141,50.141,0.393,0.393,Canadian SWD
45.0,10524.416,0.617,-52.117,52.117,-1.958,1.958,Canadian SWD
46.0,11546.145,0.646,-40.794,40.794,-0.245,0.245,Canadian SWD
47.0,11589.054,0.646,-43.812,43.812,-1.311,1.311,Canadian SWD
48.0,12559.329,0.677,-33.869,33.869,-0.663,0.663,Canadian SWD
49.0,12591.722,0.678,-37.104,37.104,-3.663,3.663,Canadian SWD
50.0,13714.657,0.710,-25.656,25.656,0.005,0.005,Canadian SWD
51.0,13744.485,0.713,-29.039,29.039,-2.020,2.020,Canadian SWD


Calumet SWD


,Flow Rate,kWh/BBL,perc_from_BEP,abs_perc_from_BEP,norm_perc_from_BEP,abs_norm_perc_from_BEP,site_id
45.0,11035.465,0.596,-37.378,37.378,NaN,NaN,Calumet SWD
46.0,11484.407,0.611,-33.941,33.941,NaN,NaN,Calumet SWD
48.0,11896.518,0.666,-34.825,34.825,-2.157,2.157,Calumet SWD
49.0,12193.108,0.686,-33.774,33.774,-2.213,2.213,Calumet SWD
50.0,10537.047,0.787,-62.313,62.313,-12.017,12.017,Calumet SWD
51.0,12086.107,0.764,-42.164,42.164,-2.738,2.738,Calumet SWD
52.0,11523.383,0.815,-53.751,53.751,-3.025,3.025,Calumet SWD
53.0,14561.142,0.750,-12.303,12.303,-0.490,0.490,Calumet SWD
54.0,13368.489,0.819,-33.221,33.221,-47.669,47.669,Calumet SWD
55.0,14273.366,0.834,-23.472,23.472,-9.471,9.471,Calumet SWD
